In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch
import evaluate

# SST2 Dataset

In [ ]:
# Load dataset
dataset = load_dataset("glue", "sst2")

# Convert to text-to-text format
def preprocess(example):
    label_text = "positive" if example["label"] == 1 else "negative"
    return {
        "input_text": "sst2 sentence: " + example["sentence"],
        "target_text": label_text
    }

dataset = dataset.map(preprocess)

# Load tokenizer
model_name = "google/switch-base-8"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    model_inputs = tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=4)
    model_inputs["label"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


# QQP Dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset("glue", "qqp")

def preprocess(example):
    label_text = "duplicate" if example["label"] == 1 else "not duplicate"
    return {
        "input_text": f"qqp question1: {example['question1']} question2: {example['question2']}",
        "target_text": label_text
    }

dataset = dataset.map(preprocess)


from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/switch-base-8")

def tokenize(example):
    model_inputs = tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["target_text"], padding="max_length", truncation=True, max_length=4)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
label_map = {"not duplicate": 0, "duplicate": 1}
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = [p for p in preds if all(0 <= token < tokenizer.vocab_size for token in p)]
    labels = [l for l in labels if all(0 <= token < tokenizer.vocab_size for token in l)]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print("Decoded Predictions:", decoded_preds[:10])
    print("Decoded Labels:", decoded_labels[:10])


    decoded_preds = [p.strip().lower() for p in decoded_preds]
    decoded_labels = [l.strip().lower() for l in decoded_labels]

    label_map = {"not duplicate": 0, "duplicate": 1}
    mapped_preds = [label_map.get(p, -1) for p in decoded_preds]
    mapped_labels = [label_map.get(l, -1) for l in decoded_labels]

    filtered = [(p, l) for p, l in zip(mapped_preds, mapped_labels) if p != -1 and l != -1]

    if len(filtered) == 0:
        return {"accuracy": 0.0}  # or skip logging

    mapped_preds, mapped_labels = zip(*filtered)

    return metric.compute(predictions=mapped_preds, references=mapped_labels)



In [ ]:
# Accuracy metric


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     decoded_preds = [p.strip().lower() for p in decoded_preds]
#     decoded_labels = [l.strip().lower() for l in decoded_labels]
#     return metric.compute(predictions=decoded_preds, references=decoded_labels)
import os
os.makedirs("/content/drive/MyDrive/sst2-switch", exist_ok=True)
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/sst2-switch",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=5,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(30000)),  # use subset for faster training
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,
)

<ipython-input-35-8230dd5ab470>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.082700,0.060608,0.939220
2,0.059800,0.067243,0.934633
3,0.042500,0.064803,0.932339
4,0.040500,0.076288,0.930046
5,0.028300,0.086305,0.928899


Decoded Predictions: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Labels: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Predictions: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Labels: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Predictions: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Labels: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Predictions: ['positive', 'negative', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative']
Decoded Labels: ['pos

TrainOutput(global_step=18750, training_loss=0.31740577829996747, metrics={'train_runtime': 4955.5525, 'train_samples_per_second': 30.269, 'train_steps_per_second': 3.784, 'total_flos': 6.85052854272e+16, 'train_loss': 0.31740577829996747, 'epoch': 5.0})

Epoch,Training Loss,Validation Loss
0,11.448900,0.087608


{'eval_loss': 0.08760783821344376}

In [ ]:
model.save_pretrained("/content/drive/MyDrive/sst2-switch-model")
tokenizer.save_pretrained("/content/drive/MyDrive/sst2-switch-tokenizer")

('/content/drive/MyDrive/sst2-switch-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/sst2-switch-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/sst2-switch-tokenizer/spiece.model',
 '/content/drive/MyDrive/sst2-switch-tokenizer/added_tokens.json',
 '/content/drive/MyDrive/sst2-switch-tokenizer/tokenizer.json')